## Setup Aplikasi Deteksi Objek YOLOv8 dengan Flask di Google Colab

Notebook ini akan mengkonfigurasi dan menjalankan aplikasi Flask yang menggunakan model YOLOv8 untuk deteksi objek. Aplikasi akan diekspos ke internet menggunakan `ngrok`.

**Prasyarat:**
1.  **Akun Google & Google Drive:** Untuk menyimpan file proyek dan model.
2.  **Akun Ngrok & Authtoken:** Untuk mengekspos server Flask. Dapatkan dari dashboard.ngrok.com.
3.  **Database MySQL:** Aplikasi memerlukan koneksi ke server MySQL. Pastikan Anda memiliki kredensial dan host yang dapat diakses dari Colab.
4.  **File Proyek di Google Drive:**
    *   Buat folder proyek di Google Drive Anda (misalnya, `MyDrive/projek-yolo8_colab/`).
    *   Unggah direktori `app` (berisi `templates` dan `static`) dan `models_yolo` (berisi model `.pt`) ke folder proyek tersebut.
    *   Buat file `.env` di dalam folder proyek di Drive dengan konfigurasi yang diperlukan (API key, secret key, path model, kredensial DB). Lihat contoh di deskripsi output.
5.  **ESP32-CAM:** Pastikan kamera ESP32-CAM Anda aktif dan dapat dijangkau oleh Colab (IP address yang dimasukkan di dashboard harus dapat diakses oleh server Colab).

In [ ]:
!pip install flask flask-ngrok pyngrok mysql-connector-python werkzeug google-generativeai python-dotenv requests ultralytics opencv-python-headless numpy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Konfigurasi Path Proyek dan Ngrok Authtoken

1.  Sesuaikan `PROJECT_ROOT` dengan path ke folder proyek Anda di Google Drive.
2.  Masukkan `NGROK_AUTHTOKEN` Anda.

In [ ]:
import os

# 1. Sesuaikan dengan path folder proyek Anda di Google Drive
PROJECT_ROOT = '/content/drive/MyDrive/projek-yolo8_colab/' 

# Pastikan direktori proyek ada
if not os.path.exists(PROJECT_ROOT):
    print(f"Error: Direktori proyek '{PROJECT_ROOT}' tidak ditemukan. Harap periksa path dan pastikan folder sudah ada di Google Drive Anda.")
else:
    print(f"PROJECT_ROOT diatur ke: {PROJECT_ROOT}")
    # Buat folder uploads jika belum ada di dalam app/static di Drive
    uploads_path = os.path.join(PROJECT_ROOT, 'app/static/uploads')
    os.makedirs(uploads_path, exist_ok=True)
    print(f"Folder uploads dipastikan ada/dibuat di: {uploads_path}")

# 2. Masukkan NGROK Authtoken Anda di sini
NGROK_AUTHTOKEN = "YOUR_NGROK_AUTHTOKEN" # GANTI DENGAN AUTHTOKEN NGROK ANDA

if NGROK_AUTHTOKEN == "YOUR_NGROK_AUTHTOKEN":
    print("PERINGATAN: Harap ganti 'YOUR_NGROK_AUTHTOKEN' dengan authtoken ngrok Anda yang sebenarnya.")
else:
    # Konfigurasi ngrok authtoken
    os.system(f"ngrok authtoken {NGROK_AUTHTOKEN}")
    print("Ngrok authtoken telah dikonfigurasi.")

# Path ke file .env di Google Drive
DOTENV_PATH = os.path.join(PROJECT_ROOT, '.env')
if not os.path.exists(DOTENV_PATH):
    print(f"PERINGATAN: File .env tidak ditemukan di {DOTENV_PATH}. Pastikan Anda telah membuatnya sesuai instruksi.")
else:
    print(f"File .env akan dimuat dari: {DOTENV_PATH}")

### Kode Aplikasi Utama
Sel berikut berisi kode utama aplikasi Flask yang diadaptasi dari `app_run.py`.

In [ ]:
import os
from flask import Flask, render_template, request, redirect, url_for, session, flash, Response as FlaskResponse
import mysql.connector
from werkzeug.security import generate_password_hash, check_password_hash # type: ignore
from functools import wraps
from datetime import datetime
import google.generativeai as genai
from typing import Optional, Dict, Any, Tuple, Union, Callable, cast
from dotenv import load_dotenv
import requests
from ultralytics import YOLO
import cv2
import base64
import time
import numpy as np
import threading
from werkzeug.wrappers import Response as WerkzeugResponse
from mysql.connector.connection import MySQLConnection
from mysql.connector.pooling import PooledMySQLConnection
from mysql.connector.abstracts import MySQLConnectionAbstract
from flask_ngrok import run_with_ngrok # Untuk integrasi ngrok

# Muat variabel lingkungan dari file .env di Google Drive
load_dotenv(dotenv_path=DOTENV_PATH)

# Inisialisasi Aplikasi Flask dengan path template dan static dari Google Drive
app = Flask(__name__, 
            template_folder=os.path.join(PROJECT_ROOT, 'app/templates'), 
            static_folder=os.path.join(PROJECT_ROOT, 'app/static'))

# Konfigurasi Google Gemini API
GEMINI_API_KEY = os.getenv("GOOGLE_GEMINI_API_KEY")
if not GEMINI_API_KEY:
    app.logger.warning("GOOGLE_GEMINI_API_KEY tidak ditemukan di .env. Fitur deskripsi Gemini tidak akan berfungsi.")
else:
    try:
        genai.configure(api_key=GEMINI_API_KEY) # type: ignore
        app.logger.info("Google Gemini API berhasil dikonfigurasi.")
    except Exception as e:
        app.logger.error(f"Error saat mengkonfigurasi Gemini API: {e}")
        GEMINI_API_KEY = None

# Konfigurasi Aplikasi
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', os.urandom(24).hex())
app.config['UPLOAD_FOLDER'] = os.path.join(PROJECT_ROOT, 'app/static/uploads') # Path di Google Drive
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

# Muat Model YOLOv8 sekali saat aplikasi dimulai
default_model_oil_path = os.path.join(PROJECT_ROOT, 'models_yolo', 'best.pt') # Path di Google Drive
MODEL_PATH_OIL = os.getenv('YOLO_MODEL_OIL_PATH', default_model_oil_path)
if not os.path.isabs(MODEL_PATH_OIL): # Jika path di .env relatif, jadikan absolut ke PROJECT_ROOT
    MODEL_PATH_OIL = os.path.join(PROJECT_ROOT, MODEL_PATH_OIL)

model_yolo_oil: Optional[YOLO] = None
try:
    if os.path.exists(MODEL_PATH_OIL):
        model_yolo_oil = YOLO(MODEL_PATH_OIL) # type: ignore
        app.logger.info(f"Model YOLO untuk OLI berhasil dimuat dari {MODEL_PATH_OIL}")
    else:
        app.logger.error(f"Error: File model YOLO untuk OLI tidak ditemukan di path: {MODEL_PATH_OIL}")
except Exception as e:
    app.logger.error(f"Error saat memuat model YOLO untuk OLI dari {MODEL_PATH_OIL}: {e}")
    model_yolo_oil = None

# Konfigurasi Database MySQL (mengambil dari .env)
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_USER = os.getenv("DB_USER", "root")
DB_PASSWORD = os.getenv("DB_PASSWORD", "")
DB_NAME = os.getenv("DB_NAME", "db_projek_yolo8")

app.logger.info(f"Menggunakan konfigurasi DB: Host={DB_HOST}, User={DB_USER}, DB={DB_NAME}")
if DB_HOST == "localhost" or DB_HOST == "your_mysql_host_address":
    app.logger.warning("PERINGATAN: DB_HOST masih 'localhost' atau placeholder. Pastikan ini dikonfigurasi dengan benar di file .env untuk koneksi ke server MySQL Anda yang dapat diakses dari Colab.")

# Konfigurasi Kamera
CAMERA_REQUEST_TIMEOUT = int(os.getenv("CAMERA_REQUEST_TIMEOUT", "60"))
CAMERA_VERIFY_TIMEOUT = int(os.getenv("CAMERA_VERIFY_TIMEOUT", "30"))
CAMERA_STREAM_PATH = "/stream"
CAMERA_CAPTURE_PATH = "/stream"

def get_camera_base_ip() -> Optional[str]:
    camera_ip = session.get('esp32_cam_ip')
    if camera_ip:
        if not camera_ip.startswith(('http://', 'https://')):
            return f"http://{camera_ip}"
        return camera_ip
    return None

def verify_camera_connection(ip_address: str) -> Tuple[bool, str]:
    if not ip_address:
        return False, "Alamat IP kosong."
    if not ip_address.startswith(('http://', 'https://')):
        base_url = f"http://{ip_address}"
    else:
        base_url = ip_address
    verify_url = f"{base_url}{CAMERA_STREAM_PATH}"
    app.logger.info(f"Verifying camera connection to: {verify_url} with timeout {CAMERA_VERIFY_TIMEOUT}s")
    try:
        with requests.get(verify_url, timeout=CAMERA_VERIFY_TIMEOUT, stream=True) as response:
            if response.status_code == 200:
                 try:
                     chunk = next(response.iter_content(chunk_size=1024), None)
                     if chunk is not None:
                         app.logger.info(f"Verification successful for {ip_address}. Received initial data from stream.")
                         return True, "Koneksi stream berhasil diverifikasi."
                     else:
                         app.logger.warning(f"Verification failed for {ip_address}: Stream responded with 200 OK but no initial data was received.")
                         return False, "Koneksi stream berhasil (200 OK), tetapi tidak ada data stream awal."
                 except requests.exceptions.RequestException as e:
                      app.logger.error(f"Verification error for {ip_address}: Failed to read initial data - {e}")
                      return False, f"Gagal membaca data stream awal: {e}"
            else:
                app.logger.warning(f"Verification failed for {ip_address}: Status {response.status_code}")
                return False, f"Server stream merespons: {response.status_code}"
    except requests.exceptions.Timeout:
        app.logger.warning(f"Verification timeout for {ip_address}")
        return False, f"Timeout ({CAMERA_VERIFY_TIMEOUT}s) saat verifikasi."
    except requests.exceptions.RequestException as e:
        app.logger.error(f"Verification connection error for {ip_address}: {e}")
        return False, f"Gagal terhubung ke stream: {e}"
    except Exception as e:
        app.logger.error(f"Unexpected verification error for {ip_address}: {e}")
        return False, f"Error tak terduga: {e}"

app.logger.info(f"  Path Stream Kamera: {CAMERA_STREAM_PATH}")
app.logger.info(f"  Path Capture Kamera: {CAMERA_CAPTURE_PATH}")
app.logger.info(f"  Timeout Request Kamera: {CAMERA_REQUEST_TIMEOUT}s")
app.logger.info(f"  Timeout Verifikasi Kamera: {CAMERA_VERIFY_TIMEOUT}s")

def capture_single_frame_from_http_endpoint(capture_url: str, timeout: int = 10) -> Tuple[Optional[np.ndarray], Optional[str]]:
    try:
        app.logger.info(f"Mencoba mengambil gambar dari HTTP endpoint: {capture_url} timeout {timeout}s")
        response = requests.get(capture_url, timeout=timeout)
        response.raise_for_status()
        image_bytes = response.content
        if not image_bytes: return None, "Tidak ada data gambar."
        image_np = cv2.imdecode(np.frombuffer(image_bytes, np.uint8), cv2.IMREAD_COLOR)
        if image_np is None: return None, "Gagal decode gambar."
        return image_np, None
    except requests.exceptions.Timeout: return None, f"Timeout ({timeout}s) mengambil gambar."
    except requests.exceptions.HTTPError as http_err: return None, f"HTTP error: {http_err}"
    except requests.exceptions.RequestException as req_err: return None, f"Kesalahan koneksi: {req_err}"
    except Exception as e: return None, f"Kesalahan tak terduga: {str(e)}"

opencv_ffmpeg_options_lock = threading.Lock()

def capture_single_frame_from_stream_cv2(stream_url: str, read_frame_timeout: int = 10, open_stream_timeout_sec: int = 60) -> Tuple[Optional[np.ndarray], Optional[str]]:
    ffmpeg_timeout_us = str(open_stream_timeout_sec * 1000 * 1000)
    ffmpeg_options_to_set = f"timeout;{ffmpeg_timeout_us}|rw_timeout;{ffmpeg_timeout_us}"
    cap: Optional[cv2.VideoCapture] = None
    original_ffmpeg_options_env: Optional[str] = None
    stream_opened_successfully = False
    with opencv_ffmpeg_options_lock:
        original_ffmpeg_options_env = os.environ.get("OPENCV_FFMPEG_CAPTURE_OPTIONS")
        os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = ffmpeg_options_to_set
        try:
            max_open_attempts = 3
            for attempt in range(max_open_attempts):
                app.logger.info(f"Membuka stream OpenCV: {stream_url} (Attempt {attempt + 1})")
                current_attempt_cap = cv2.VideoCapture(stream_url, cv2.CAP_FFMPEG)
                if current_attempt_cap.isOpened():
                    cap = current_attempt_cap
                    stream_opened_successfully = True
                    break
                else:
                    current_attempt_cap.release()
                    if attempt < max_open_attempts - 1: time.sleep(1.5)
        finally:
            if original_ffmpeg_options_env is None:
                if "OPENCV_FFMPEG_CAPTURE_OPTIONS" in os.environ: del os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"]
            else:
                os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = original_ffmpeg_options_env
    if not stream_opened_successfully or cap is None:
        return None, f"Gagal membuka stream {stream_url} setelah {max_open_attempts} percobaan."
    try:
        start_time = time.time()
        frame: Optional[np.ndarray] = None; ret = False
        for i in range(2):
            if time.time() - start_time > read_frame_timeout: break
            temp_ret, temp_frame = cap.read()
            if temp_ret and temp_frame is not None: ret = True; frame = temp_frame
            else: time.sleep(0.2)
        if not ret or frame is None: return None, f"Gagal membaca frame dari {stream_url}."
        return frame, None
    finally:
        if cap is not None: cap.release()

@app.template_filter('date')
def custom_date_filter(value: Union[str, datetime], fmt: Optional[str] = None) -> str:
    format_string = fmt if fmt else "%Y-%m-%d %H:%M:%S"
    if fmt == "Y": format_string = "%Y"
    if value == "now": return datetime.utcnow().strftime(format_string)
    if isinstance(value, datetime): return value.strftime(format_string)
    if isinstance(value, str):
        try: return datetime.fromisoformat(value).strftime(format_string)
        except ValueError: pass
    return str(value)

def get_db_connection() -> Optional[Union[MySQLConnection, MySQLConnectionAbstract, PooledMySQLConnection]]:
    try:
        conn = mysql.connector.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, database=DB_NAME, connect_timeout=10)
        return conn
    except mysql.connector.Error as err:
        flash(f"Kesalahan koneksi database: {err}", "danger")
        app.logger.error(f"Database connection error: {err}")
        return None

def login_required(f: Callable) -> Callable:
    @wraps(f)
    def decorated_function(*args: Any, **kwargs: Any) -> Any:
        if 'user_id' not in session:
            flash('Anda harus login untuk mengakses halaman ini.', 'warning')
            return redirect(url_for('login'))
        return f(*args, **kwargs)
    return decorated_function

@app.route('/')
def index() -> WerkzeugResponse:
    if 'user_id' in session: return redirect(url_for('dashboard'))
    return redirect(url_for('login'))

@app.route('/register', methods=['GET', 'POST'])
def register() -> Union[str, WerkzeugResponse]:
    if request.method == 'POST':
        username = request.form.get('username', '').strip()
        email = request.form.get('email', '').strip()
        password = request.form.get('password', '')
        confirm_password = request.form.get('confirm_password', '')
        if not all([username, email, password, confirm_password]):
            flash('Semua field wajib diisi!', 'danger'); return redirect(url_for('register'))
        if password != confirm_password:
            flash('Password tidak cocok!', 'danger'); return redirect(url_for('register'))
        conn = get_db_connection()
        if not conn: return render_template('register.html', title="Register", error_message="DB connection error.")
        cursor = None
        try:
            cursor = conn.cursor(dictionary=True)
            cursor.execute("SELECT * FROM users WHERE username = %s OR email = %s", (username, email))
            if cursor.fetchone():
                flash('Username/Email sudah terdaftar.', 'warning'); return redirect(url_for('register'))
            hashed_password = generate_password_hash(password)
            cursor.execute("INSERT INTO users (username, email, password_hash) VALUES (%s, %s, %s)", (username, email, hashed_password))
            conn.commit()
            flash('Registrasi berhasil! Silakan login.', 'success'); return redirect(url_for('login'))
        except mysql.connector.Error as err:
            flash(f'Error registrasi: {err}', 'danger')
            if conn.is_connected(): conn.rollback()
            return redirect(url_for('register'))
        finally:
            if cursor: cursor.close()
            if conn and conn.is_connected(): conn.close()
    return render_template('register.html', title="Register")

@app.route('/login', methods=['GET', 'POST'])
def login() -> Union[str, WerkzeugResponse]:
    if request.method == 'POST':
        identifier = request.form.get('identifier', '').strip()
        password = request.form.get('password', '')
        if not identifier or not password:
            flash('Username/Email dan Password wajib diisi!', 'danger'); return redirect(url_for('login'))
        conn = get_db_connection()
        if not conn: return render_template('login.html', title="Login", error_message="DB connection error.")
        cursor = None
        try:
            cursor = conn.cursor(dictionary=True)
            cursor.execute("SELECT id, username, password_hash FROM users WHERE username = %s OR email = %s", (identifier, identifier))
            user: Optional[Dict[str, Any]] = cursor.fetchone() # type: ignore[assignment]
            if user and check_password_hash(user['password_hash'], password):
                session['user_id'] = user['id']; session['username'] = user['username']
                flash(f"Selamat datang, {str(user['username'])}!", 'success'); return redirect(url_for('dashboard'))
            else:
                flash('Login gagal. Periksa kredensial Anda.', 'danger'); return redirect(url_for('login'))
        except mysql.connector.Error as err:
            flash(f'Error login: {err}', 'danger'); return redirect(url_for('login'))
        finally:
            if cursor: cursor.close()
            if conn and conn.is_connected(): conn.close()
    return render_template('login.html', title="Login")

@app.route('/dashboard')
@login_required
def dashboard() -> str:
    camera_base_ip = get_camera_base_ip()
    esp32_stream_url = f"{camera_base_ip}{CAMERA_STREAM_PATH}" if camera_base_ip else None
    if not camera_base_ip: flash("IP Kamera belum dikonfigurasi.", "warning")
    return render_template('dashboard.html', title="Dashboard", username=session.get('username'), 
                           current_cam_ip=session.get('esp32_cam_ip', ""), 
                           esp32_stream_url=esp32_stream_url, camera_configured=(camera_base_ip is not None))

@app.route('/update_cam_ip', methods=['POST'])
@login_required
def update_cam_ip() -> WerkzeugResponse:
    new_cam_ip = request.form.get('esp32_cam_ip', '').strip()
    if new_cam_ip:
        if '.' not in new_cam_ip and len(new_cam_ip) <= 3: # Validasi sederhana
             flash('Format IP Kamera tidak valid.', 'danger'); return redirect(url_for('dashboard'))
        is_verified, verify_message = verify_camera_connection(new_cam_ip)
        if is_verified:
            session['esp32_cam_ip'] = new_cam_ip
            flash(f'IP Kamera diperbarui & diverifikasi: {new_cam_ip}', 'success')
        else:
            session['esp32_cam_ip'] = new_cam_ip # Simpan meski gagal verifikasi
            flash(f'IP Kamera diperbarui: {new_cam_ip}. PERINGATAN: Verifikasi gagal - {verify_message}', 'warning')
    else:
        session.pop('esp32_cam_ip', None)
        flash('IP Kamera dihapus dari sesi.', 'info')
    return redirect(url_for('dashboard'))

def get_gemini_description(image_path_for_gemini: str, detected_class_name: str) -> str:
    if not GEMINI_API_KEY or not genai: return "Deskripsi Gemini tidak tersedia (API tidak dikonfigurasi)."
    try:
        model_gemini = genai.GenerativeModel('gemini-1.5-flash-latest') # type: ignore
        if not os.path.exists(image_path_for_gemini): return f"Gagal: File gambar tidak ditemukan ({os.path.basename(image_path_for_gemini)})."
        image_input = genai.upload_file(image_path_for_gemini) # type: ignore
        prompt = (
            f"Analisis gambar ini. Model deteksi objek mengidentifikasi objek utama sebagai '{detected_class_name}'.\n\n"
            f"Berdasarkan visual dan identifikasi '{detected_class_name}':\n"
            f"1. Jelaskan objek dan konteksnya, terkait '{detected_class_name}'.\n"
            f"2. Jika '{detected_class_name}' terkait kondisi oli motor, berikan analisis kualitas oli dan saran perawatan.\n"
            f"3. Jika bukan oli, berikan deskripsi umum yang relevan.\n\n"
            f"PENTING: Jawaban ringkas, maks 2-3 paragraf, tiap paragraf maks 8 kalimat."
        )
        response = model_gemini.generate_content([prompt, image_input])
        return response.text if response.text else "Tidak ada teks dari Gemini."
    except Exception as e:
        app.logger.error(f"Error Gemini API: {e}")
        return f"Gagal menghasilkan deskripsi Gemini: {e}"

def _process_image_data_and_save_detection(original_image_np: np.ndarray, user_id: int, upload_folder: str, gemini_api_key_present: bool) -> Tuple[bool, str, Optional[int]]:
    if model_yolo_oil is None: return False, "Model YOLO oli tidak dimuat.", None
    try:
        timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
        base_image_name = f"capture_{user_id}_{timestamp_str}"
        original_image_filename = f"{base_image_name}_original.jpg"
        absolute_original_image_path = os.path.join(upload_folder, original_image_filename)
        annotated_image_filename = f"{base_image_name}_annotated.jpg"
        absolute_annotated_image_path = os.path.join(upload_folder, annotated_image_filename)
        relative_annotated_image_path = f"uploads/{annotated_image_filename}" # Untuk URL dan DB

        if not cv2.imwrite(absolute_original_image_path, original_image_np): return False, "Gagal simpan gambar asli.", None
        annotated_image_to_save = original_image_np.copy()
        all_detection_details = []; all_detected_class_names = []
        class_for_gemini_description: Optional[str] = None
        db_detected_class_name = "Tidak ada objek terdeteksi"; db_confidence_score_str =""

        if model_yolo_oil:
            results_oil = model_yolo_oil(original_image_np, verbose=False)
            if results_oil and results_oil[0].boxes:
                annotated_image_to_save = results_oil[0].plot(img=annotated_image_to_save, conf=True, labels=True)
                for i, box in enumerate(results_oil[0].boxes):
                    class_name = model_yolo_oil.names.get(int(box.cls[0].item()), "UnknownOil")
                    confidence = float(box.conf[0].item())
                    all_detection_details.append(f"{class_name}: {confidence*100:.2f}% (Oli)")
                    all_detected_class_names.append(class_name)
                    if i == 0 and not class_for_gemini_description: class_for_gemini_description = class_name

        if all_detected_class_names:
            db_detected_class_name = ", ".join(sorted(list(set(all_detected_class_names))))
            if all_detection_details: db_confidence_score_str = ", ".join(all_detection_details)

        if not cv2.imwrite(absolute_annotated_image_path, annotated_image_to_save): 
            return False, "Gagal simpan gambar anotasi.", None

        generative_desc = "Tidak ada objek terdeteksi/klasifikasi."
        if gemini_api_key_present and class_for_gemini_description:
            generative_desc = get_gemini_description(absolute_original_image_path, class_for_gemini_description)
        elif not gemini_api_key_present: generative_desc = "Fitur Gemini tidak aktif."
        # ... (logika deskripsi lainnya bisa ditambahkan jika perlu)

        db_conn = None; cursor = None
        try:
            db_conn = get_db_connection()
            if not db_conn: return False, "Gagal koneksi DB untuk simpan deteksi.", None
            cursor = db_conn.cursor()
            sql = "INSERT INTO detections (user_id, image_name, image_path, detection_class, confidence_score, generative_description, timestamp) VALUES (%s, %s, %s, %s, %s, %s, %s)"
            val = (user_id, annotated_image_filename, relative_annotated_image_path, db_detected_class_name, db_confidence_score_str, generative_desc, datetime.now())
            cursor.execute(sql, val)
            db_conn.commit()
            return True, "Deteksi berhasil diproses.", cursor.lastrowid
        except mysql.connector.Error as db_err:
            if db_conn and db_conn.is_connected(): db_conn.rollback()
            return False, f"Gagal simpan ke DB: {db_err}", None
        finally:
            if cursor: cursor.close()
            if db_conn and db_conn.is_connected(): db_conn.close()
    except Exception as e:
        app.logger.error(f"Error proses gambar: {e}", exc_info=True)
        return False, f"Error tak terduga proses gambar: {e}", None

@app.route('/process_browser_capture', methods=['POST'])
@login_required
def process_browser_capture() -> Tuple[Dict[str, Any], int]:
    if not model_yolo_oil: return {"status": "error", "message": "Model YOLO oli tidak dimuat."}, 503
    data = request.get_json()
    if not data or 'image_data_url' not in data: return {"status": "error", "message": "Data gambar tidak ada."}, 400
    try:
        _, encoded_data = data['image_data_url'].split(',', 1)
        image_np = cv2.imdecode(np.frombuffer(base64.b64decode(encoded_data), np.uint8), cv2.IMREAD_COLOR)
        if image_np is None: return {"status": "error", "message": "Gagal decode base64."}, 400
        success, message, detection_id = _process_image_data_and_save_detection(
            image_np, session['user_id'], app.config['UPLOAD_FOLDER'], bool(GEMINI_API_KEY)
        )
        if success and detection_id is not None:
            return {"status": "success", "redirect_url": url_for('hasil', detection_id=detection_id), "message": message}, 200
        return {"status": "error", "message": message or "Gagal proses gambar."}, 500
    except Exception as e:
        app.logger.error(f"Error /process_browser_capture: {e}", exc_info=True)
        return {"status": "error", "message": f"Error server: {str(e)}"}, 500

@app.route('/get_snapshot_for_canvas')
@login_required
def get_snapshot_for_canvas() -> Tuple[Dict[str, Any], int]:
    camera_base_ip = get_camera_base_ip()
    if not camera_base_ip: return {"status": "error", "message": "IP Kamera tidak dikonfigurasi."}, 400
    snapshot_url = f"{camera_base_ip}{str(CAMERA_CAPTURE_PATH)}"
    if str(CAMERA_CAPTURE_PATH) == CAMERA_STREAM_PATH:
        image_np, error_msg = capture_single_frame_from_stream_cv2(snapshot_url, open_stream_timeout_sec=20, read_frame_timeout=10)
    else:
        image_np, error_msg = capture_single_frame_from_http_endpoint(snapshot_url, timeout=CAMERA_REQUEST_TIMEOUT)
    if error_msg or image_np is None: return {"status": "error", "message": error_msg or "Gagal ambil snapshot."}, 502
    try:
        is_success, buffer = cv2.imencode(".jpg", image_np)
        if not is_success: return {"status": "error", "message": "Gagal encode JPEG."}, 500
        image_data_url = f"data:image/jpeg;base64,{base64.b64encode(buffer.tobytes()).decode('utf-8')}"
        return {"status": "success", "image_data_url": image_data_url}, 200
    except Exception as e:
        return {"status": "error", "message": f"Error proses snapshot: {str(e)}"}, 500

@app.route('/uji_kamera')
@login_required
def uji_kamera_page() -> str:
    camera_base_ip = get_camera_base_ip()
    stream_url_for_template = f"{camera_base_ip}{CAMERA_STREAM_PATH}" if camera_base_ip else None
    display_ip_for_template = camera_base_ip.replace("http://", "").replace("https://", "") if camera_base_ip else "Kamera tidak dikonfigurasi"
    if not camera_base_ip: flash("IP Kamera belum dikonfigurasi.", "warning")
    return render_template('index.html', title="Uji Capture Kamera", username=session.get('username'),
                           esp32_stream_url_from_flask=stream_url_for_template,
                           esp32_display_ip_from_flask=display_ip_for_template)

@app.route('/api/capture_and_process', methods=['POST'])
@login_required
def api_capture_and_process() -> Union[FlaskResponse, WerkzeugResponse, Tuple[Dict[str, Any], int]]:
    camera_base_ip = get_camera_base_ip()
    if not camera_base_ip: return {"error": "IP Kamera belum dikonfigurasi."}, 503
    if not model_yolo_oil: return {"error": "Model YOLO oli tidak dimuat."}, 503
    single_image_capture_url = f"{camera_base_ip}{str(CAMERA_CAPTURE_PATH)}"
    try:
        img_np: Optional[np.ndarray] = None; error_msg: Optional[str] = None
        if str(CAMERA_CAPTURE_PATH) == CAMERA_STREAM_PATH:
            img_np, error_msg = capture_single_frame_from_stream_cv2(single_image_capture_url, open_stream_timeout_sec=20, read_frame_timeout=10)
        else:
            img_np, error_msg = capture_single_frame_from_http_endpoint(single_image_capture_url, timeout=CAMERA_REQUEST_TIMEOUT)
        if error_msg or img_np is None: return {"error": f"Gagal ambil gambar: {error_msg}"}, 502
        try:
            annotated_image_bgr_np = img_np.copy()
            if model_yolo_oil:
                results_oil = model_yolo_oil(img_np, verbose=False)
                if results_oil and results_oil[0].boxes:
                    annotated_image_bgr_np = results_oil[0].plot(img=annotated_image_bgr_np, conf=True, labels=True)
            encode_success, image_buffer = cv2.imencode('.jpg', annotated_image_bgr_np)
            if not encode_success: return {"error": "Gagal encode anotasi."}, 500
            return FlaskResponse(image_buffer.tobytes(), mimetype='image/jpeg')
        except Exception as yolo_err:
            return {"error": f"Error YOLO: {str(yolo_err)}"}, 500
    except Exception as e:
        return {"error": f"Error internal server: {str(e)}"}, 500

@app.route('/hasil/<int:detection_id>')
@login_required
def hasil(detection_id: int) -> Union[str, WerkzeugResponse]:
    conn = get_db_connection(); cursor = None; detection_data: Optional[Dict[str, Any]] = None
    if conn:
        try:
            cursor = conn.cursor(dictionary=True)
            cursor.execute("SELECT * FROM detections WHERE id = %s AND user_id = %s", (detection_id, session['user_id']))
            detection_data = cursor.fetchone() # type: ignore[assignment]
        except mysql.connector.Error as err: flash(f"Error ambil data: {err}", "danger")
        finally:
            if cursor: cursor.close()
            if conn.is_connected(): conn.close()
    if not detection_data: flash("Data tidak ditemukan.", "warning"); return redirect(url_for('dashboard'))
    return render_template('hasil.html', title="Hasil Deteksi", username=session.get('username'), detection=detection_data)

@app.route('/histori')
@login_required
def histori() -> str:
    detections_history: list = []; conn = get_db_connection(); cursor = None
    if conn:
        try:
            cursor = conn.cursor(dictionary=True)
            cursor.execute("SELECT * FROM detections WHERE user_id = %s ORDER BY timestamp DESC", (session['user_id'],))
            detections_history = cursor.fetchall()
        except mysql.connector.Error as err: flash(f"Error muat histori: {err}", "danger")
        finally:
            if cursor: cursor.close()
            if conn.is_connected(): conn.close()
    return render_template('histori.html', title="Histori Deteksi", username=session.get('username'), detections=detections_history)

@app.route('/hapus_deteksi/<int:detection_id>', methods=['GET'])
@login_required
def hapus_deteksi(detection_id: int) -> WerkzeugResponse:
    conn = None; cursor = None
    try:
        conn = get_db_connection()
        if not conn: return redirect(url_for('histori'))
        cursor = conn.cursor(dictionary=True)
        cursor.execute("SELECT image_name FROM detections WHERE id = %s AND user_id = %s", (detection_id, session['user_id']))
        detection_row: Optional[Dict[str, Any]] = cursor.fetchone() # type: ignore[assignment]
        if not detection_row: flash('Riwayat tidak ditemukan.', 'warning'); return redirect(url_for('histori'))
        
        image_name_from_db = str(detection_row['image_name'])
        annotated_image_path = os.path.join(app.config['UPLOAD_FOLDER'], image_name_from_db)
        if image_name_from_db.endswith("_annotated.jpg"):
            original_image_path = os.path.join(app.config['UPLOAD_FOLDER'], image_name_from_db.replace("_annotated.jpg", "_original.jpg"))
            if os.path.exists(original_image_path): os.remove(original_image_path)
        if os.path.exists(annotated_image_path): os.remove(annotated_image_path)

        cursor.execute("DELETE FROM detections WHERE id = %s AND user_id = %s", (detection_id, session['user_id']))
        conn.commit()
        flash('Riwayat berhasil dihapus.', 'success')
    except (mysql.connector.Error, OSError) as err:
        flash(f'Gagal hapus: {err}', 'danger')
        if conn and conn.is_connected(): conn.rollback()
    finally:
        if cursor: cursor.close()
        if conn and conn.is_connected(): conn.close()
    return redirect(url_for('histori'))

@app.route('/logout')
@login_required
def logout() -> WerkzeugResponse:
    session.clear(); flash('Anda telah logout.', 'success'); return redirect(url_for('login'))

# Error Handlers
@app.errorhandler(400)
def bad_request(e: Any) -> Tuple[str, int]: return render_template('errors/400.html', title="Permintaan Buruk"), 400
@app.errorhandler(401)
def unauthorized(e: Any) -> Tuple[str, int]: 
    flash("Sesi tidak valid atau perlu login.", "warning")
    return render_template('errors/401.html', title="Tidak Diotorisasi"), 401
@app.errorhandler(403)
def forbidden(e: Any) -> Tuple[str, int]: return render_template('errors/403.html', title="Terlarang"), 403
@app.errorhandler(404)
def page_not_found(e: Any) -> Tuple[str, int]: return render_template('errors/404.html', title="Tidak Ditemukan"), 404
@app.errorhandler(405)
def method_not_allowed(e: Any) -> Tuple[str, int]: return render_template('errors/405.html', title="Metode Salah"), 405
@app.errorhandler(500)
def internal_server_error(e: Any) -> Tuple[str, int]:
    app.logger.error(f"Internal Server Error: {e}", exc_info=True)
    return render_template('errors/500.html', title="Kesalahan Server"), 500


### Menjalankan Aplikasi Flask dengan Ngrok
Sel berikut akan menjalankan aplikasi Flask dan mengeksposnya menggunakan `ngrok`. URL publik akan ditampilkan di output.

In [ ]:
if NGROK_AUTHTOKEN == "YOUR_NGROK_AUTHTOKEN" or not NGROK_AUTHTOKEN:
  print("ERROR: NGROK_AUTHTOKEN tidak diatur. Harap set di sel konfigurasi di atas.")
elif not os.path.exists(PROJECT_ROOT) or not os.path.isdir(PROJECT_ROOT):
  print(f"ERROR: PROJECT_ROOT '{PROJECT_ROOT}' tidak valid atau tidak ditemukan. Harap periksa konfigurasinya.")
elif not os.path.exists(os.path.join(PROJECT_ROOT, 'app/templates')):
  print(f"ERROR: Folder template tidak ditemukan di {os.path.join(PROJECT_ROOT, 'app/templates')}. Pastikan struktur proyek benar di Drive.")
elif DB_HOST == "your_mysql_host_address" or DB_USER == "your_mysql_username":
  print(f"PERINGATAN: Konfigurasi database (DB_HOST, DB_USER) tampaknya masih menggunakan placeholder. Pastikan file .env di '{DOTENV_PATH}' sudah diisi dengan benar.")
  print("Aplikasi akan tetap dicoba dijalankan, tetapi kemungkinan akan gagal pada operasi database.")
  run_with_ngrok(app) # type: ignore
  app.run()
else:
  print(f"Memulai Flask app dengan ngrok...")
  print(f"Template folder: {app.template_folder}")
  print(f"Static folder: {app.static_folder}")
  print(f"Upload folder: {app.config['UPLOAD_FOLDER']}")
  print(f"Model Oli Path: {MODEL_PATH_OIL}")
  run_with_ngrok(app) # type: ignore
  app.run()